In [1]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# Launch MAPDL
mapdl = launch_mapdl(loglevel="ERROR", exec_file="C:/Program Files/ANSYS Inc/v232/ansys/bin/winx64/ANSYS232.exe", nproc=10,)
mapdl.clear()
mapdl.prep7()
mapdl.units("SI")  # Use SI units

# --- Define Materials ---
# Material 1: Concrete (temperature-dependent)
mapdl.mp("DENS", 1, 2400)
mapdl.mptemp(1, 100)
mapdl.mptemp(2, 150)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 300)
mapdl.mptemp(5, 400)
mapdl.mptemp(6, 500)
mapdl.mpdata("C", 1, 1, 900)
mapdl.mpdata("C", 1, 2, 950)
mapdl.mpdata("C", 1, 3, 1000)
mapdl.mpdata("C", 1, 4, 1050)
mapdl.mpdata("C", 1, 5, 1100)
mapdl.mpdata("C", 1, 6, 1100)
mapdl.mpdata("KXX", 1, 1, 2.5)
mapdl.mpdata("KXX", 1, 2, 2.417)
mapdl.mpdata("KXX", 1, 3, 2.333)
mapdl.mpdata("KXX", 1, 4, 2.167)
mapdl.mpdata("KXX", 1, 5, 2.0)
mapdl.mpdata("KXX", 1, 6, 2.0)

# Material 2: Structural Steel (assumed constant properties)
mapdl.mp("DENS", 2, 7850)
mapdl.mp("C", 2, 434)
mapdl.mp("KXX", 2, 60.5)

# --- Geometry Parameters ---
od_media = 0.254     # Outer diameter of media (m)
od_hx = 0.0508       # Outer diameter of HX pipe (m)
t_hx = 0.00635       # Wall thickness of pipe (m)
id_hx = od_hx - 2 * t_hx  # Inner diameter of HX pipe (m)
length = 0.508       # Cylinder height
num_radial = 40
num_circ = 18

# --- Create Keypoints ---
k0 = mapdl.k("", 0, 0, 0)
inner_kps = []
outer_kps = []
for i in range(num_circ):
    angle = i * (360 / num_circ)
    x_inner = (id_hx / 2) * np.cos(np.deg2rad(angle))
    y_inner = (id_hx / 2) * np.sin(np.deg2rad(angle))
    x_outer = (od_media / 2) * np.cos(np.deg2rad(angle))
    y_outer = (od_media / 2) * np.sin(np.deg2rad(angle))
    inner_kps.append(mapdl.k("", x_inner, y_inner, 0))
    outer_kps.append(mapdl.k("", x_outer, y_outer, 0))

# --- Create Lines and Areas ---
inner_arcs = [mapdl.l(inner_kps[i], inner_kps[(i+1)%num_circ]) for i in range(num_circ)]
outer_arcs = [mapdl.l(outer_kps[i], outer_kps[(i+1)%num_circ]) for i in range(num_circ)]
radial_lines = [mapdl.l(inner_kps[i], outer_kps[i]) for i in range(num_circ)]
areas = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    lines = [inner_arcs[i], radial_lines[next_i], outer_arcs[i], radial_lines[i]]
    areas.append(mapdl.al(*lines))

# --- Set element type and mesh controls ---
mapdl.et(1, "SOLID70")
volumes = [mapdl.vext(a, dz=length) for a in areas]

# Set radial mesh size
mapdl.lsel("S", "LINE", "", min(radial_lines), max(radial_lines))
mapdl.lesize("ALL", "", "", num_radial, 1)

# Set axial mesh size
num_z = 10
mapdl.lsel("S", "LOC", "X", 0)
mapdl.lsel("R", "LOC", "Y", 0)
mapdl.lsel("R", "LOC", "Z", 0, length)
mapdl.lesize("ALL", "", "", num_z, 1)
mapdl.allsel()
mapdl.vmesh("ALL")
mapdl.smrtsize(2)

# --- Plot Mesh ---
mapdl.eplot(vtk=True, show_edges=True, line_width=1)



C:\Users\arnold\AppData\Local\Temp\ipykernel_20532\1007861958.py:88: DeprecationWarning: From 0.70.0, the arguments 'vtk' and 'use_vtk' are deprecated. To use interactive plots, use 'graphics_backend=GraphicsBackend.PYVISTA' instead. 
  mapdl.eplot(vtk=True, show_edges=True, line_width=1)


[82, 87, 110]


Widget(value='<iframe src="http://localhost:62876/index.html?ui=P_0x15c640ade80_0&reconnect=auto" class="pyvis…

In [2]:
mapdl.finish()
mapdl.slashsolu()

import matplotlib.pyplot as plt

# Set Up Transient Analysis
mapdl.antype(4)  # Transient thermal analysis
mapdl.trnopt("FULL")  # Full transient solution
mapdl.kbc(0)  # Ramped loading (smooth boundary conditions)
mapdl.timint("ON")  # Enable time integration

# Define initial condition (uniform temperature)
mapdl.ic("ALL", "TEMP", 100)  # Initial temperature of 100°C

# Define time steps
time_steps = np.linspace(0, 14400, 145)  # 0 to 14400 seconds in 24 steps

temp_load = np.linspace(100, 400, len(time_steps))  # 每一步温度线性上升

for i, t in enumerate(time_steps):
    if i == 0:
        continue  # skip initial step

    mapdl.time(t)


    inner_radius = id_hx / 2  # 0.01905 m
    points = mapdl.mesh.grid.points
    radii = np.sqrt(points[:, 0]**2 + points[:, 1]**2)

    final_node_indices = np.where(np.abs(radii - inner_radius) < 1e-3)[0]
    final_nodes = final_node_indices + 1  # +1，因为Python是从0计数

    mapdl.nsel("NONE")
    for node in final_nodes:
        mapdl.nsel("A", "NODE", "", node)



#    if i == 1:
#       mapdl.nplot(vtk=True, show_node_numbering=True)


    mapdl.d("ALL", "TEMP", temp_load[i])


    mapdl.nsel("ALL")



    mapdl.autots("ON")
    mapdl.deltim(100)

    mapdl.solve()

mapdl.finish()
mapdl.post1()

*****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

          ***** MAPDL RESULTS INTERPRETATION (POST1) *****

 *** NOTE ***                            CP =       0.000   TIME= 00:00:00
 Reading results into the database (SET command) will update the current 
 displacement and force boundary conditions in the database with the     
 values from the results file for that load set.  Note that any          
 subsequent solutions will use these values unless action is taken to    
 either SAVE the current values or not overwrite them (/EXIT,NOSAVE).    


 USE LAST SUBSTEP ON RESULT FILE  FOR LOAD CASE 0 

 SET COMMAND GOT LOAD STEP=   144  SUBSTEP=     1  CUMULATIVE ITERATION=   144
   TIME/FREQUENCY=  14400.    
 TITLE=

In [3]:
mapdl.finish()
mapdl.post1()

# 尝试通过 XYZ 直接找最近节点
inner_radius = id_hx / 2
outer_radius = od_media / 2

# Select nearest to (inner_radius, 0, z_center)
mapdl.nsel("S", "LOC", "X", inner_radius - 1e-3, inner_radius + 1e-3)
mapdl.nsel("R", "LOC", "Y", -1e-3, 1e-3)
mapdl.nsel("R", "LOC", "Z", -1e-3, 1e-3)
inner_node = int(mapdl.get("INNER", "NODE", 0, "NUM", "MAX"))

# Select nearest to (outer_radius, 0, z_center)
mapdl.nsel("S", "LOC", "X", outer_radius - 1e-3, outer_radius + 1e-3)
mapdl.nsel("R", "LOC", "Y", -1e-3, 1e-3)
mapdl.nsel("R", "LOC", "Z", -1e-3, 1e-3)
outer_node = int(mapdl.get("OUTER", "NODE", 0, "NUM", "MAX"))

mapdl.allsel()
mapdl.finish()

print(f"inner_node = {inner_node}, outer_node = {outer_node}")


inner_node = 1, outer_node = 45


In [4]:
grid = mapdl.mesh.grid  # PyVista-compatible mesh
nodes = mapdl.mesh.nodes
coords = grid.points  # Nx3 array of (x, y, z)


for nid, coord in zip(nodes, coords):
    print(f"Node {nid}: x={coord[0]:.4f}, y={coord[1]:.4f}, z={coord[2]:.4f}")


Node [0.01905 0.      0.     ]: x=0.0190, y=0.0000, z=0.0000
Node [0.01790114 0.00651548 0.        ]: x=0.0179, y=0.0065, z=0.0000
Node [0.01866705 0.00217183 0.        ]: x=0.0187, y=0.0022, z=0.0000
Node [0.0182841  0.00434366 0.        ]: x=0.0183, y=0.0043, z=0.0000
Node [0.11934096 0.04343656 0.        ]: x=0.1193, y=0.0434, z=0.0000
Node [0.02043714 0.00743851 0.        ]: x=0.0204, y=0.0074, z=0.0000
Node [0.02297314 0.00836154 0.        ]: x=0.0230, y=0.0084, z=0.0000
Node [0.02550913 0.00928456 0.        ]: x=0.0255, y=0.0093, z=0.0000
Node [0.02804513 0.01020759 0.        ]: x=0.0280, y=0.0102, z=0.0000
Node [0.03058112 0.01113062 0.        ]: x=0.0306, y=0.0111, z=0.0000
Node [0.03311712 0.01205364 0.        ]: x=0.0331, y=0.0121, z=0.0000
Node [0.03565311 0.01297667 0.        ]: x=0.0357, y=0.0130, z=0.0000
Node [0.03818911 0.0138997  0.        ]: x=0.0382, y=0.0139, z=0.0000
Node [0.0407251  0.01482273 0.        ]: x=0.0407, y=0.0148, z=0.0000
Node [0.0432611  0.01574575 0

In [5]:
import os

mapdl.post1()
num_steps = len(time_steps)
inner_temps = []
outer_temps = []
for i in range(1, num_steps):
    mapdl.set(i)
    temp_inner = mapdl.get("TIN", "NODE", inner_node, "TEMP")
    temp_outer = mapdl.get("TOUT", "NODE", outer_node, "TEMP")
    inner_temps.append(temp_inner)
    outer_temps.append(temp_outer)

import pandas as pd

df = pd.DataFrame({
    "Time (s)": time_steps[1:],  # 从 step 1 开始
    "Inner Temp (°C)": inner_temps,
    "Outer Temp (°C)": outer_temps,
})

df.to_csv("temperature_points.csv", index=False)
print("Saved temperature record to 'temperature_points.csv'")

Saved temperature record to 'temperature_points.csv'
